<a href="https://colab.research.google.com/github/spacewal/cap/blob/main/Data_Mining_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install yfinance package
!pip install yfinance

In [11]:
import yfinance as yf
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [12]:
# Fetch S&P 500 tickers
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500_tickers = table[0]['Symbol'].tolist()
sp500_tickers = [ticker.replace('.', '-') for ticker in sp500_tickers]

In [13]:
# Initialize an empty list to store the data
data = []

# Loop through each ticker symbol
for ticker in sp500_tickers:
    # Fetch the ticker data
    stock = yf.Ticker(ticker)

    # Get the info and extract 'previousClose' and 'volume'
    info = stock.info
    previous_close = info.get('previousClose', None)
    volume = info.get('volume', None)

    # Append a dictionary to the data list
    data.append({
        'Ticker': ticker,
        'PreviousClose': previous_close,
        'Volume': volume
    })

# Convert the list of dictionaries into a DataFrame
df_stocks = pd.DataFrame(data)

In [14]:
df_stocks.head()

,Ticker,PreviousClose,Volume
0,MMM,92.61,3377257
1,AOS,86.47,1002766
2,ABT,112.44,3965374
3,ABBV,170.14,2423150
4,ACN,333.16,2155364


In [15]:
df_stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ticker         503 non-null    object 
 1   PreviousClose  503 non-null    float64
 2   Volume         503 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.9+ KB


In [16]:
df_volume_greater_than_mil = df_stocks[df_stocks['Volume'] > 1000000]

In [17]:
df_volume_greater_than_mil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 364 entries, 0 to 502
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Ticker         364 non-null    object 
 1   PreviousClose  364 non-null    float64
 2   Volume         364 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.4+ KB


In [ ]:
# Define the Ichimoku Cloud calculation function
def calculate_ichimoku_cloud(df):
    tenkan_period = 9
    kijun_period = 26
    senkou_span_b_period = 52

    df['conversion_line'] = (df['High'].rolling(window=tenkan_period).max() + df['Low'].rolling(window=tenkan_period).min()) / 2
    df['base_line'] = (df['High'].rolling(window=kijun_period).max() + df['Low'].rolling(window=kijun_period).min()) / 2
    df['senkou_span_a'] = ((df['conversion_line'] + df['base_line']) / 2).shift(kijun_period)
    df['senkou_span_b'] = ((df['High'].rolling(window=senkou_span_b_period).max() + df['Low'].rolling(window=senkou_span_b_period).min()) / 2).shift(kijun_period)

    return df

# Define the function to check if the last price is above the Ichimoku Cloud
def check_above_cloud(last_price, span_a, span_b):
    if last_price >= span_a and last_price >= span_b:
        return "ABOVE CLOUD"
    else:
        return "NOT ABOVE CLOUD"

# Fetch historical data for each stock and apply the Ichimoku Cloud calculation
for ticker in df_volume_greater_than_mil['Ticker']:
    stock_data = yf.download(ticker, period="max")
    stock_data_ichimoku = calculate_ichimoku_cloud(stock_data)

    # Assuming 'last_price' is the last 'Close' price from the historical data
    last_price = stock_data['Close'].iloc[-1]
    span_a = stock_data['senkou_span_a'].iloc[-1]
    span_b = stock_data['senkou_span_b'].iloc[-1]

    # Check if the last price is above the Ichimoku Cloud and assign the result to the 'CloudStatus' column
    cloud_status = check_above_cloud(last_price, span_a, span_b)

    # Find the index of the current ticker in 'df_filtered'
    index = df_volume_greater_than_mil[df_volume_greater_than_mil['Ticker'] == ticker].index[0]

    # Assign the cloud status to the new column 'CloudStatus' for the corresponding row in 'df_filtered'
    df_volume_greater_than_mil.at[index, 'CloudStatus'] = cloud_status


In [19]:
df_volume_greater_than_mil.head()

,Ticker,PreviousClose,Volume,CloudStatus
0,MMM,92.61,3377257,ABOVE CLOUD
1,AOS,86.47,1002766,ABOVE CLOUD
2,ABT,112.44,3965374,NOT ABOVE CLOUD
3,ABBV,170.14,2423150,NOT ABOVE CLOUD
4,ACN,333.16,2155364,NOT ABOVE CLOUD


In [ ]:
# Define the Awesome Oscillator calculation function
def calculate_awesome_oscillator(df, short_period=5, long_period=34):
    # Calculate the midpoint ((High + Low) / 2) of each bar
    df['Midpoint'] = (df['High'] + df['Low']) / 2

    # Calculate the short and long period SMAs of the midpoints
    df['SMA_Short'] = df['Midpoint'].rolling(window=short_period).mean()
    df['SMA_Long'] = df['Midpoint'].rolling(window=long_period).mean()

    # Calculate the Awesome Oscillator
    df['AO'] = df['SMA_Short'] - df['SMA_Long']

    return df

# Assuming df_filtered is your DataFrame with the filtered tickers
# Initialize a list to store the Awesome Oscillator values
ao_values = []

# Fetch historical data for each stock and calculate the Awesome Oscillator
for ticker in df_volume_greater_than_mil['Ticker']:
    # Download historical stock data
    stock_data = yf.download(ticker, period='max')

    # Calculate Awesome Oscillator
    stock_data_with_ao = calculate_awesome_oscillator(stock_data)

    # Get the last Awesome Oscillator value
    ao_last_value = stock_data_with_ao['AO'].iloc[-1]

    # Append to the list of AO values
    ao_values.append(ao_last_value)

# Add the AO values as a new column to df_filtered
df_volume_greater_than_mil['Awesome_Oscillator'] = ao_values


In [21]:
df_volume_greater_than_mil.head()

,Ticker,PreviousClose,Volume,CloudStatus,Awesome_Oscillator
0,MMM,92.61,3377257,ABOVE CLOUD,7.265581
1,AOS,86.47,1002766,ABOVE CLOUD,0.873265
2,ABT,112.44,3965374,NOT ABOVE CLOUD,-4.549206
3,ABBV,170.14,2423150,NOT ABOVE CLOUD,-7.746649
4,ACN,333.16,2155364,NOT ABOVE CLOUD,-29.410853


In [ ]:
# Define the interpretation functions
def interpret_ao(ao_value):
    return "BULLISH" if ao_value >= 0 else "BEARISH"

def interpret_ao_movement(current_ao, previous_ao):
    if current_ao >= 0 and previous_ao < current_ao:
        return "BULLISH_INCREASING"
    elif current_ao >= 0 and previous_ao > current_ao:
        return "BULLISH_DECREASING"
    elif current_ao < 0 and previous_ao < current_ao:
        return "BEARISH_INCREASING"
    elif current_ao < 0 and previous_ao > current_ao:
        return "BEARISH_DECREASING"
    return "STABLE"  # If current and previous AO values are the same

# Initialize lists to store the interpretation results
ao_interpretation_list = []
ao_movement_list = []

# Fetch historical data for each stock and calculate indicators
for ticker in df_volume_greater_than_mil['Ticker']:
    # Download historical stock data
    stock_data = yf.download(ticker, period='max')

    # Calculate Awesome Oscillator
    stock_data_with_ao = calculate_awesome_oscillator(stock_data)

    # Get the last two Awesome Oscillator values
    if len(stock_data_with_ao['AO']) >= 2:
        current_ao = stock_data_with_ao['AO'].iloc[-1]
        previous_ao = stock_data_with_ao['AO'].iloc[-2]
    else:
        # Handle cases where there are not enough data points
        current_ao = previous_ao = None  # or use np.nan to denote missing values

    # Interpret the AO value and movement
    ao_interpretation = interpret_ao(current_ao) if current_ao is not None else None
    ao_movement = interpret_ao_movement(current_ao, previous_ao) if None not in (current_ao, previous_ao) else None

    # Append the interpretations to the lists
    ao_interpretation_list.append(ao_interpretation)
    ao_movement_list.append(ao_movement)

# Add the interpretation and movement columns to df_filtered
df_volume_greater_than_mil['AO_Interpretation'] = ao_interpretation_list
df_volume_greater_than_mil['AO_Movement'] = ao_movement_list


In [23]:
df_volume_greater_than_mil.head()

,Ticker,PreviousClose,Volume,CloudStatus,Awesome_Oscillator,AO_Interpretation,AO_Movement
0,MMM,92.61,3377257,ABOVE CLOUD,7.265581,BULLISH,BULLISH_DECREASING
1,AOS,86.47,1002766,ABOVE CLOUD,0.873265,BULLISH,BULLISH_DECREASING
2,ABT,112.44,3965374,NOT ABOVE CLOUD,-4.549206,BEARISH,BEARISH_INCREASING
3,ABBV,170.14,2423150,NOT ABOVE CLOUD,-7.746649,BEARISH,BEARISH_DECREASING
4,ACN,333.16,2155364,NOT ABOVE CLOUD,-29.410853,BEARISH,BEARISH_DECREASING


In [ ]:
# Define the VWAP calculation function
def calculate_vwap(df):
    df['VWAP'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
    return df

# Initialize a list to store the VWAP values
vwap_values = []

# Fetch historical data for each stock and calculate VWAP
for ticker in df_volume_greater_than_mil['Ticker']:
    # Download historical stock data
    stock_data = yf.download(ticker, period='max')

    # Calculate VWAP
    stock_data_with_vwap = calculate_vwap(stock_data)

    # Get the last VWAP value
    vwap_last_value = stock_data_with_vwap['VWAP'].iloc[-1]

    # Append to the list of VWAP values
    vwap_values.append(vwap_last_value)

# Add the VWAP values as a new column to df_filtered
df_volume_greater_than_mil['VWAP'] = vwap_values


In [25]:
df_volume_greater_than_mil.head()

,Ticker,PreviousClose,Volume,CloudStatus,Awesome_Oscillator,AO_Interpretation,AO_Movement,VWAP
0,MMM,92.61,3377257,ABOVE CLOUD,7.265581,BULLISH,BULLISH_DECREASING,63.143570
1,AOS,86.47,1002766,ABOVE CLOUD,0.873265,BULLISH,BULLISH_DECREASING,23.721173
2,ABT,112.44,3965374,NOT ABOVE CLOUD,-4.549206,BEARISH,BEARISH_INCREASING,26.436008
3,ABBV,170.14,2423150,NOT ABOVE CLOUD,-7.746649,BEARISH,BEARISH_DECREASING,86.114643
4,ACN,333.16,2155364,NOT ABOVE CLOUD,-29.410853,BEARISH,BEARISH_DECREASING,92.802848


In [26]:
# Define the function to calculate EMA using pandas 'ewm' method
def calculate_ema(series, span):
    return series.ewm(span=span, adjust=False).mean()

# Fetch historical data for each stock and calculate EMAs
for ticker in df_volume_greater_than_mil['Ticker']:
    # Download historical stock data
    stock_data = yf.download(ticker, period='max')

    # Calculate each EMA and its percent difference from the close price
    for window in [20, 36, 50, 95, 200]:
        ema_column_name = f'EMA_{window}'
        stock_data[ema_column_name] = calculate_ema(stock_data['Close'], span=window)
        stock_data[f'{ema_column_name}_Percent'] = ((stock_data['Close'] - stock_data[ema_column_name]) / stock_data['Close'] * 100).round(2)

    # Get the last values of each EMA and its percent difference
    for window in [20, 36, 50, 95, 200]:
        ema_column_name = f'EMA_{window}'
        df_volume_greater_than_mil.loc[df_volume_greater_than_mil['Ticker'] == ticker, ema_column_name] = stock_data[ema_column_name].iloc[-1]
        df_volume_greater_than_mil.loc[df_volume_greater_than_mil['Ticker'] == ticker, f'{ema_column_name}_Percent'] = stock_data[f'{ema_column_name}_Percent'].iloc[-1]

# df_filtered now has the EMA columns and their percent differences


[*********************100%%**********************]  1 of 1 completed
<ipython-input-26-0d316655038e>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_volume_greater_than_mil.loc[df_volume_greater_than_mil['Ticker'] == ticker, ema_column_name] = stock_data[ema_column_name].iloc[-1]
<ipython-input-26-0d316655038e>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_volume_greater_than_mil.loc[df_volume_greater_than_mil['Ticker'] == ticker, f'{ema_column_name}_Percent'] = stock_data[f'{ema_column_name}_Pe

In [27]:
df_volume_greater_than_mil.head()

,Ticker,PreviousClose,Volume,CloudStatus,Awesome_Oscillator,AO_Interpretation,AO_Movement,VWAP,EMA_20,EMA_20_Percent,EMA_36,EMA_36_Percent,EMA_50,EMA_50_Percent,EMA_95,EMA_95_Percent,EMA_200,EMA_200_Percent
0,MMM,92.61,3377257,ABOVE CLOUD,7.265581,BULLISH,BULLISH_DECREASING,63.143570,89.078032,3.83,86.735776,6.36,85.679803,7.50,84.369299,8.92,85.220077,8.00
1,AOS,86.47,1002766,ABOVE CLOUD,0.873265,BULLISH,BULLISH_DECREASING,23.721173,86.991285,-1.71,85.826472,-0.35,84.787269,0.87,81.896650,4.25,77.267315,9.66
2,ABT,112.44,3965374,NOT ABOVE CLOUD,-4.549206,BEARISH,BEARISH_INCREASING,26.436008,112.777290,-1.45,113.531180,-2.12,113.375127,-1.98,111.534894,-0.33,108.827608,2.11
3,ABBV,170.14,2423150,NOT ABOVE CLOUD,-7.746649,BEARISH,BEARISH_DECREASING,86.114643,175.155734,-3.62,175.009895,-3.54,173.544350,-2.67,167.922664,0.66,160.154593,5.25
4,ACN,333.16,2155364,NOT ABOVE CLOUD,-29.410853,BEARISH,BEARISH_DECREASING,92.802848,344.627381,-6.24,352.158070,-8.56,354.073600,-9.15,350.483267,-8.04,335.962859,-3.56


In [ ]:
# Define the function to calculate smoothed RSI
def calculate_rsi(data, periods=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=periods).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=periods).mean()

    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))

    return RSI

# Define the function to calculate traditional RSI
def calculate_rsi_Trad(data, period=14):
    delta = data.diff(1)
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    average_gain = gain.rolling(window=period).mean()
    average_loss = loss.rolling(window=period).mean()

    rs = average_gain / average_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Fetch historical data for each stock and calculate RSIs
for ticker in df_volume_greater_than_mil['Ticker']:
    # Download historical stock data
    stock_data = yf.download(ticker, period='max')

    # Ensure there is enough data to calculate RSI
    if len(stock_data) > 14:
        # Calculate Smoothed RSI and Traditional RSI
        stock_data['RSI_Smoothed'] = calculate_rsi(stock_data['Close'])
        stock_data['RSI_Trad'] = calculate_rsi_Trad(stock_data['Close'])

        # Get the last values of each RSI
        last_rsi_smoothed = stock_data['RSI_Smoothed'].iloc[-1]
        last_rsi_trad = stock_data['RSI_Trad'].iloc[-1]
    else:
        last_rsi_smoothed = None
        last_rsi_trad = None

    # Assign the last RSI values to the corresponding ticker in df_filtered
    df_volume_greater_than_mil.loc[df_volume_greater_than_mil['Ticker'] == ticker, 'RSI_Smoothed'] = last_rsi_smoothed
    df_volume_greater_than_mil.loc[df_volume_greater_than_mil['Ticker'] == ticker, 'RSI_Trad'] = last_rsi_trad

# df_filtered now has the two new RSI columns


In [29]:
df_volume_greater_than_mil.head()

,Ticker,PreviousClose,Volume,CloudStatus,Awesome_Oscillator,AO_Interpretation,AO_Movement,VWAP,EMA_20,EMA_20_Percent,EMA_36,EMA_36_Percent,EMA_50,EMA_50_Percent,EMA_95,EMA_95_Percent,EMA_200,EMA_200_Percent,RSI_Smoothed,RSI_Trad
0,MMM,92.61,3377257,ABOVE CLOUD,7.265581,BULLISH,BULLISH_DECREASING,63.143570,89.078032,3.83,86.735776,6.36,85.679803,7.50,84.369299,8.92,85.220077,8.00,55.868091,55.868091
1,AOS,86.47,1002766,ABOVE CLOUD,0.873265,BULLISH,BULLISH_DECREASING,23.721173,86.991285,-1.71,85.826472,-0.35,84.787269,0.87,81.896650,4.25,77.267315,9.66,40.186044,40.186044
2,ABT,112.44,3965374,NOT ABOVE CLOUD,-4.549206,BEARISH,BEARISH_INCREASING,26.436008,112.777290,-1.45,113.531180,-2.12,113.375127,-1.98,111.534894,-0.33,108.827608,2.11,48.792972,48.792972
3,ABBV,170.14,2423150,NOT ABOVE CLOUD,-7.746649,BEARISH,BEARISH_DECREASING,86.114643,175.155734,-3.62,175.009895,-3.54,173.544350,-2.67,167.922664,0.66,160.154593,5.25,34.777121,34.777121
4,ACN,333.16,2155364,NOT ABOVE CLOUD,-29.410853,BEARISH,BEARISH_DECREASING,92.802848,344.627381,-6.24,352.158070,-8.56,354.073600,-9.15,350.483267,-8.04,335.962859,-3.56,19.536855,19.536855


In [ ]:
# Define the cahold function
def cahold(previous_close, latest_price):
    return "BULLISH" if latest_price >= previous_close else "BEARISH"

# Add a new column to df_filtered for the cahold values
cahold_values = []

# Fetch historical data for each stock
for ticker in df_volume_greater_than_mil['Ticker']:
    # Download historical stock data
    stock_data = yf.download(ticker, period='2d')  # Fetch only the last two days

    # Check if we got at least two days of data to compare the latest and previous close prices
    if len(stock_data) >= 2:
        # Get the latest and previous close prices
        latest_price = stock_data['Close'].iloc[-1]
        previous_close = stock_data['Close'].iloc[-2]

        # Calculate the cahold value
        cahold_value = cahold(previous_close, latest_price)
    else:
        # In case there's not enough data, we can't determine the cahold value
        cahold_value = None

    # Append the cahold value to the list
    cahold_values.append(cahold_value)

# Assign the cahold values to the new column in df_filtered
df_volume_greater_than_mil['Cahold_Status'] = cahold_values


In [31]:
df = df_volume_greater_than_mil

In [3]:
df.tail()

NameError: name 'df' is not defined

In [33]:
# Set the option to display all columns (None means unlimited)
pd.set_option('display.max_columns', None)

# Set the option to display all rows (None means unlimited)
pd.set_option('display.max_rows', None)

# Assuming 'df' is your DataFrame
print(df.head())

  Ticker  PreviousClose   Volume      CloudStatus  Awesome_Oscillator  \
0    MMM          92.61  3377257      ABOVE CLOUD            7.265581   
1    AOS          86.47  1002766      ABOVE CLOUD            0.873265   
2    ABT         112.44  3965374  NOT ABOVE CLOUD           -4.549206   
3   ABBV         170.14  2423150  NOT ABOVE CLOUD           -7.746649   
4    ACN         333.16  2155364  NOT ABOVE CLOUD          -29.410853   

  AO_Interpretation         AO_Movement       VWAP      EMA_20  \
0           BULLISH  BULLISH_DECREASING  63.143570   89.078032   
1           BULLISH  BULLISH_DECREASING  23.721173   86.991285   
2           BEARISH  BEARISH_INCREASING  26.436008  112.777290   
3           BEARISH  BEARISH_DECREASING  86.114643  175.155734   
4           BEARISH  BEARISH_DECREASING  92.802848  344.627381   

   EMA_20_Percent      EMA_36  EMA_36_Percent      EMA_50  EMA_50_Percent  \
0            3.83   86.735776            6.36   85.679803            7.50   
1         

In [34]:
# Output the DataFrame to a CSV file
df.to_csv('full_dataframe.csv', index=False)

# comprehensive market analysis and trend forecasting

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
# Filter the data for the MMM ticker
mmm_data = df[df['Ticker'] == 'MMM']

NameError: name 'df' is not defined